In [85]:
# Import necessary libraries
#from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import random

In [86]:
random_wait = random.randint(2, 4)

#Red flags según artículo
flags_auto = ("conversable", "al dia", "detalles", "atrasada", "atrasado", "atrasados", "trizado", "trizada", "choques", "choque", "chocado", "deshuase", "desarme", "le falta", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "no transferible", "multa", "multas", "parte", "partes")

flags = flags_auto

close_locations = ["Valparaiso", "Viña del Mar"]

In [87]:
login = False #True da más detalles de los autos pero puede ser problemático, False da menos detalles pero es más robusto

# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 500000
max_price = 10000000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
min_km = 10
max_km = 300000
min_year = 1990
max_year = 2024
transmission = "manual"
make = "Mazda "
search_model = ""
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [88]:
# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Configurar las opciones de Chrome
options = Options()
# options.add_argument("--headless")  # Para ejecutar Chrome en modo sin cabeza (sin ventana)
options.add_experimental_option("detach", True) # Para que el driver no se cierre al terminar el script
# deshabilitar notificaciones
options.add_argument("--disable-notifications")

# Inicializar el driver de Chrome
driver = webdriver.Chrome(service=service, options=options)
# Set up base url
base_url = "https://www.facebook.com/marketplace/"

#Set up full url
url = f"{base_url}{default_location}/search?availability={availability}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_since_listed}&maxMileage={max_km}&maxYear={max_year}&minMileage={min_km}&minYear={min_year}&transmissionType={transmission}&query={make}{search_model}&exact={exact}"

In [89]:
# Inicio de sesión o no

# Cuenta fake
mail = "roadtonostromo1@gmail.com"
password = "suufreejvrrr"

if login:

    driver.get(base_url)

    mail_input = driver.find_element(By.XPATH ,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[3]/div/label/div/div/input')

    ActionChains(driver).click(mail_input).perform()

    mail_input.send_keys(mail)

    password_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[4]/div/label/div/div/input')

    ActionChains(driver).click(password_input).perform()

    password_input.send_keys(password)

    mail_input.send_keys(Keys.RETURN)

    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

else:
    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

    # Buscar el elemento de la X para ignorar el inicio de sesión
    x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")

    # Hacer clic en la X para ignorar el inicio de sesión
    x_button.click()

Obtener información de la zona de búsqueda

In [90]:
change_location_selector = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]')

# Obtener ubicación actual
current_location = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]/div[1]/div/span').text
print(f"La búsqueda se está realizando en: {current_location}")

# Hacer clic en el selector de ubicación
# change_location_selector.click()

# Cambiar la ubicación de búsqueda
# location = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/div/div/label/div/div[2]/input')

# ActionChains(driver).click(location).perform()

# esperar 1 segundo
time.sleep(random_wait)

La búsqueda se está realizando en: Curacaví, Region Metropolitana, Chile
  · En un radio de 65 kilómetros


In [91]:
#location.clear()

In [92]:
#location.send_keys(search_location)

#location.send_keys(Keys.RETURN)

# Hay que hacer que seleccione una de las opciones del menu desplegable

Facebook cambia los id de algunos elementos, cambiar el radio de busqueda como ultima alternativa ya que se debe volver a buscar el XPATH cada cierto tiempo

In [93]:
""" # buscar selector de radio
# radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label')
radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div')                                               
# Hacer clic en el selector de radio
radius_selector.click()

# Botones de seleccion de radio

elementos = driver.find_elements(By.XPATH, "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z']")

# Selecciona el radio en el menú

for elemento in elementos:
    numero = re.match(r"\d+", elemento.text).group()
    
    if int(numero) == radius:
        elemento.click()
        break

# Buscar el botón de aplicar
apply_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div')

# Hacer clic en el botón de aplicar
apply_button.click() """

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\PC\AppData\Local\Temp\ipykernel_2212\3550737752.py:1: SyntaxWarning: invalid escape sequence '\d'
  """ # buscar selector de radio


' # buscar selector de radio\n# radius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label\')\nradius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div\')                                               \n# Hacer clic en el selector de radio\nradius_selector.click()\n\n# Botones de seleccion de radio\n\nelementos = driver.find_elements(By.XPATH, "//div[@class=\'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z\']")\n\n# Selecciona el radio en el menú\n\nfor elemento in elementos:\n    num

In [94]:
""" # Desplazarse hacia abajo para cargar más resultados
scroll_count = 6  # Número de veces a desplazar la página
scroll_delay = random_wait  # Retraso entre cada desplazamiento en segundos

for _ in range(scroll_count):
    # Ejecutar JavaScript para desplazarse hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar un momento para permitir que se cargue el contenido
    time.sleep(scroll_delay) """

# Número máximo de intentos para detectar si se ha alcanzado el final de la página
max_attempts = 50
# Tiempo máximo para esperar que ocurran cambios en la posición vertical de la barra de desplazamiento
max_scroll_wait = 3  # segundos

# Función para verificar si se ha alcanzado el final de la página
def is_page_end_reached(driver):
    current_scroll_position = driver.execute_script("return window.scrollY;")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)  # Esperar un momento para que se actualice la posición de desplazamiento
    new_scroll_position = driver.execute_script("return window.scrollY;")
    return new_scroll_position == current_scroll_position

# Desplazarse hacia abajo hasta que se haya alcanzado el final de la página o se haya intentado max_attempts veces
attempt = 1
while attempt <= max_attempts:
    if is_page_end_reached(driver):
        break
    attempt += 1
    time.sleep(4)  # Retraso antes de intentar nuevamente

# Si después de los intentos la página todavía no se ha cargado completamente, espera un tiempo adicional
if attempt > max_attempts:
    WebDriverWait(driver, max_scroll_wait).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

# Ahora la página debe estar completamente cargada


In [95]:
# Obtener el HTML de la página
html = driver.page_source

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')

In [96]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_and_mileages_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
locations_and_mileages_list = [location_and_mileage.text.strip() for location_and_mileage in locations_and_mileages_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [97]:
# locations_and_mileages_list


In [98]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+) km'
mileage_pattern_miles = r'(\d+) millas'

# Initialize an empty list to store adjusted mileage entries
locations_and_mileages_list2 = []

# Iterate through the original mileage entries
for item in locations_and_mileages_list:
    # Append the current mileage entry to the adjusted list
    locations_and_mileages_list2.append(item)
    
    """ # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if pattern.match(item) and len(locations_and_mileages_list2) >= 2 and pattern.match(locations_and_mileages_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        locations_and_mileages_list2.insert(-1, '0\xa0mil km') """
    
    if not item:
        #locations_and_mileages_list2.insert(-1, '0\xa0mil km')
        locations_and_mileages_list2[-1] = item.replace('', '0')
        print("Se encontró un espacio en blanco y se rellenó con 0 km")
    
    if "\xa0mil" in item:
        locations_and_mileages_list2[-1] = item.replace("\xa0mil", "")
        item2 = item.replace("\xa0mil", "")
        #PONER EL CONVERSOR DE MILLAS A KM AQUI

        # Try to find a match for the "K km" format
        match_mileage_km = re.search(mileage_pattern_km, item2)
        
        # Try to find a match for the "K miles" format
        match_mileage_miles = re.search(mileage_pattern_miles, item2)
        
        # Check if either of the formats is found
        if match_mileage_km or match_mileage_miles:
            # If "K km" format is found, convert it to meters and append to the cleaned list
            if match_mileage_km:
                locations_and_mileages_list2[-1] = str((int(match_mileage_km.group(1)) * 1000))
            # If "K miles" format is found, convert it to meters and append to the cleaned list
            else:
                locations_and_mileages_list2[-1] = str((int(match_mileage_miles.group(1)) * 1609))
                print(f"Se encontró una distancia en K millas {int(match_mileage_miles.group(1))}, se convirtió a {int(match_mileage_miles.group(1)) * 1609} metros")

Se encontró una distancia en K millas 125, se convirtió a 201125 metros
Se encontró una distancia en K millas 274, se convirtió a 440866 metros
Se encontró una distancia en K millas 150, se convirtió a 241350 metros
Se encontró un espacio en blanco y se rellenó con 0 km
Se encontró una distancia en K millas 200, se convirtió a 321800 metros
Se encontró un espacio en blanco y se rellenó con 0 km
Se encontró un espacio en blanco y se rellenó con 0 km
Se encontró una distancia en K millas 23, se convirtió a 37007 metros
Se encontró un espacio en blanco y se rellenó con 0 km
Se encontró una distancia en K millas 20, se convirtió a 32180 metros
Se encontró una distancia en K millas 90, se convirtió a 144810 metros
Se encontró una distancia en K millas 300, se convirtió a 482700 metros
Se encontró una distancia en K millas 200, se convirtió a 321800 metros
Se encontró una distancia en K millas 83, se convirtió a 133547 metros
Se encontró una distancia en K millas 205, se convirtió a 329845 m

In [99]:
# locations_and_mileages_list2

In [100]:
# Separate locations and mileages lists

# Initialize an empty list to store cleaned values
mileage_clean = []
locations_list = []

# Iterate through the locations_and_mileages_list2 in steps of 2 to separate locations and mileage
for i in range(0, len(locations_and_mileages_list2), 2):
    # Extract location
    location_parts = locations_and_mileages_list2[i].split(',')
    if len(location_parts) > 0:
        # Clean and decode the location to handle UTF-8 characters properly
        location = location_parts[0].strip().encode('utf-8').decode('utf-8')
        locations_list.append(location)   

    # Extract mileage
    mileage_parts = locations_and_mileages_list2[i + 1].split(' ')
    if len(mileage_parts) > 0:
        mileage = mileage_parts[0]
        mileage_clean.append(int(mileage))

print("Locations:", locations_list)
print("Mileage:", mileage_clean)
print(len(locations_list))
print(len(mileage_clean))

Locations: ['Melipilla', 'Santiago', 'Colina', 'Santiago', 'La Granja', 'Ñuñoa', 'Santiago', 'Ñuñoa', 'Santiago', 'Padre Hurtado', 'Quilpué', 'Viña del Mar', 'San Bernardo', 'La Florida', 'Santiago', 'Santiago', 'Calera', 'San Felipe', 'San Antonio', 'Quillota', 'Talagante', 'El Bosque', 'Rancagua', 'Independencia', 'Santiago', 'Santiago', 'Llay Llay', 'Macul', 'Santiago', 'Viña del Mar', 'Maipú', 'Cerrillos', 'Lo Espejo', 'San Ramón', 'Quillota', 'Santiago', 'Renca', 'Santiago', 'Peñalolén', 'San Ramón', 'Puente Alto', 'Los Andes', 'Melipilla', 'La Florida', 'Los Andes', 'Rengo', 'Maule', 'Maipú', 'La Cisterna', 'Quinta Normal', 'Santiago', 'Colina', 'Linares', 'La Granja', 'Curicó', 'Santiago', 'Buin', 'Viña del Mar', 'Lo Prado', 'Cerrillos', 'Vitacura', 'La Cisterna', 'Viña del Mar', 'Quinta Normal', 'Maipú', 'Villa Alemana', 'Viña del Mar', 'Quillota', 'Puente Alto', 'Las Condes', 'San Antonio', 'Puente Alto', 'Conchalí', 'San Joaquín', 'El Bosque', 'Santiago', 'San Joaquín', 'Cerr

In [101]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
    # Agregar el precio limpio a la lista
    clean_prices_list.append(precio_limpio)

In [102]:
def get_description(url):
    driver.get(url)
    
    
    try:
        # Buscar el elemento de la X para ignorar el inicio de sesión
        x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")
        # Hacer clic en la X para ignorar el inicio de sesión
        x_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        # time.sleep(random_wait)
    except:
        pass

    try:

        ver_mas_button = driver.find_element(By.XPATH, "//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1lku1pv x1a2a7pz')]//*[contains(text(), 'Ver más')]")
        # Hacer clic en el botón "Ver más"
        ver_mas_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        

    except:
        pass
    
    html = driver.page_source
    item_soup = soup(html, 'html.parser')

    #description_spans = item_soup.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u")
    description_spans = item_soup.find('div', class_="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a")

    #Si no se encuentra la descripción, se le asigna "No description"
    if description_spans == None:
        description = "No description"
    else:
        description = description_spans.text.strip()

    words = description.split()  # Dividir la descripción en palabras
    if len(words) >= 2 and words[-2:] == ["Ver", "menos"]:
        # Si las últimas dos palabras son "Ver menos", eliminarlas
        description = " ".join(words[:-2]).strip()

    #print(description)
    time.sleep(random_wait)
    return description


La parte de filtrado de resultados obtenidos es la más demorada de todas

In [103]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(make)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    title_split = titles_list[i].split()

    # Check if the title contains the search make
    if re.search(patron, item):

        items_dict["Titel"] = item
        items_dict["Year"] = int(title_split[0])
        items_dict["Make"] = title_split[1]
        items_dict["Model"] = title_split[2]
        items_dict["Price"] = clean_prices_list[i]
        items_dict["Search Model"] = search_model
        items_dict["Transmission"] = transmission
        items_dict["Mileage"] = mileage_clean[i]
        items_dict["Location"] = locations_list[i]
        items_dict["Description"] = get_description(urls_list[i]) #La parte más lenta

        detected_flags = []            

        # Check for flags in the description
        description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas y CREO que para quitar acentos
        for flag in flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                if flag == 'detalles' and ('detalles esteticos' in description or 'detalles de pintura' in description or 'paqueños detalles' in description or 'detalles menores' in description or 'detalles minimos' in description):
                    # Si encuentra detalles menores se guarda en los flags como "detalles menores"
                    detected_flags.append('detalles menores')

                if flag == 'multa' and ('ninguna multa' in description or 'sin multa' in description):
                    continue
                elif flag == 'multas' and ('sin multas' in description or 'no tiene multas' in description or 'libre de multas' in description or 'ni multas' in description or 'no multas' in description):
                    continue
                elif flag == 'parte' and ('ningun parte' in description):
                    continue
                elif flag == 'partes' and ('sin partes' in description or 'no tiene partes' in description or 'libre de partes' in description or 'ni partes' in description or 'no partes' in description):
                    continue
                elif flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles'):
                    continue
                elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                    continue
                elif flag == 'choques' and ('sin choques' in description or 'ningun detalle' in description):
                    continue
                else:
                    detected_flags.append(flag)

        if detected_flags:
            print(f"Se encontraron los flags {detected_flags} en la siguiente descripción: {items_dict['Description']}")


        items_dict["Flags"] = detected_flags
        items_dict["URL"] = urls_list[i]

        items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")


Se encontraron los flags ['al dia'] en la siguiente descripción: buen auto fue mio durante 6 años nunca tube problemas de nada vehículo de aseguradora papeles al día no multas de tag no partes se vende solo por renovación
Se encontraron los flags ['al dia'] en la siguiente descripción: Vehiculo hace años comprado a cia de seguro. De uso familiar diario. En perfectas condiciones, todo funcionando en perfecto estado. Se puede ver en el rosal con la farfana, maipu Documentos al dia, sin partes ni multas 32.000 km reales, solo se vende por renovación
Se encontraron los flags ['al dia', 'parte'] en la siguiente descripción: 🚨 Valor publicado es el pie mínimo para el Financiamiento ✅ Estado del Auto: EXCELENTE 📑 Documentos: Al Día listo para firmar 🏦 Financiamiento: Hasta de 48 cuotas 💵 Pie Mínimo de 30% según su perfil 🚘 Recibimos su Auto en Parte de Pago 💳 Tarjeta de crédito,débito,vale vista,efectivo 📅 Agenda Hora para Test Drive 📱Contáctame: [hidden information]
Se encontraron los flags 

In [104]:
# Cerrar la sesión del navegador
#driver.quit()

In [105]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
665,2007 Mazda 3,2007,Mazda,3,4600000,,manual,240000,Lota,No description,[],https://web.facebook.com/marketplace/item/1586117505543288/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
666,2014 Mazda bt-50,2014,Mazda,bt-50,1111111,,manual,145000,Teno,Tengo todos esos repuestos de mazda,[repuestos],https://web.facebook.com/marketplace/item/408997041717655/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
667,2014 Mazda 6,2014,Mazda,6,9000000,,manual,190000,San Javier,Mazda \n6\nFull\nEn excelente estado \nA toda prueba,[],https://web.facebook.com/marketplace/item/1101141111129920/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
668,2016 Mazda 3,2016,Mazda,3,9750000,,manual,102000,Peñalolén,"🚨 Mazda3 Skyactiv 2.0 MT 2016. Financiamiento Automotriz, Tarjetas de crédito, vale vista y transferencias 💳. Precio: $9.750.000 - Año 2016. - ⁠Bencinero. - Mecánico. - Motor 2.0 - 2 dueños. - 102.000 Km. - 6 airbags. - Todo funcionando bien. - En muy buenas condiciones, uso familiar. - Todo al día. - +56990132490 - DETALLES: - ⁠Detalles de pintura típicos por año y uso. Si quieres conocer todo nuestro stock vehicular visita el siguiente link. https://www.instagram.com/jyp_inversionistas/ - Se puede visitar todos los días previa llamada telefónica en La Florida. #jypinversionistas #concesionario #Seminuevos #santiago #Mazda3.","[al dia, detalles menores]",https://web.facebook.com/marketplace/item/1637449120414109/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
669,1996 Mazda artis,1996,Mazda,artis,1600000,,manual,265000,Coronel,Vendo mazda Artis giro un metal de biela No andando Motor gira Se había ajustado hace un par de meses culata hecha de 0 mangueras de agua nuevas llantas aro 14 auto con pintura al horno sunroof velocidad crucero se encuentra en coronel,[],https://web.facebook.com/marketplace/item/765159825184308/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [106]:
items_df

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
0,2015 Mazda cx-5,2015,Mazda,cx-5,6000000,,manual,150000,Melipilla,buen auto fue mio durante 6 años nunca tube problemas de nada vehículo de aseguradora papeles al día no multas de tag no partes se vende solo por renovación,[al dia],https://web.facebook.com/marketplace/item/1146412126394668/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1,2019 Mazda 3,2019,Mazda,3,7700000,,manual,32000,Santiago,"Vehiculo hace años comprado a cia de seguro. De uso familiar diario. En perfectas condiciones, todo funcionando en perfecto estado. Se puede ver en el rosal con la farfana, maipu Documentos al dia, sin partes ni multas 32.000 km reales, solo se vende por renovación",[al dia],https://web.facebook.com/marketplace/item/1431416131096270/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2,1998 Mazda b-2200,1998,Mazda,b-2200,1700000,,manual,242000,Colina,Vehículo con revisión técnica atrazada octubre del 2023; se vende por cambio de proyecto,[],https://web.facebook.com/marketplace/item/1560998851329251/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3,2019 Mazda cx-3 r skyactive mt,2019,Mazda,cx-3,3600000,,manual,72000,Santiago,"🚨 Valor publicado es el pie mínimo para el Financiamiento ✅ Estado del Auto: EXCELENTE 📑 Documentos: Al Día listo para firmar 🏦 Financiamiento: Hasta de 48 cuotas 💵 Pie Mínimo de 30% según su perfil 🚘 Recibimos su Auto en Parte de Pago 💳 Tarjeta de crédito,débito,vale vista,efectivo 📅 Agenda Hora para Test Drive 📱Contáctame: [hidden information]","[al dia, parte]",https://web.facebook.com/marketplace/item/250782478101410/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4,2015 Mazda cx-5,2015,Mazda,cx-5,6000000,,manual,150000,La Granja,Muy buen vehiculo fue mio durante 6 años aprox... motor 2.0 150 mil km Papeles al dia es de remate valor 6 millones conversable se vende solo por renovacion interesados hablar al numero [hidden information],"[conversable, al dia]",https://web.facebook.com/marketplace/item/1649935815767880/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
...,...,...,...,...,...,...,...,...,...,...,...,...
665,2007 Mazda 3,2007,Mazda,3,4600000,,manual,240000,Lota,No description,[],https://web.facebook.com/marketplace/item/1586117505543288/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
666,2014 Mazda bt-50,2014,Mazda,bt-50,1111111,,manual,145000,Teno,Tengo todos esos repuestos de mazda,[repuestos],https://web.facebook.com/marketplace/item/408997041717655/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
667,2014 Mazda 6,2014,Mazda,6,9000000,,manual,190000,San Javier,Mazda \n6\nFull\nEn excelente estado \nA toda prueba,[],https://web.facebook.com/marketplace/item/1101141111129920/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
668,2016 Mazda 3,2016,Mazda,3,9750000,,manual,102000,Peñalolén,"🚨 Mazda3 Skyactiv 2.0 MT 2016. Financiamiento Automotriz, Tarjetas de crédito, vale vista y transferencias 💳. Precio: $9.750.000 - Año 2016. - ⁠Bencinero. - Mecánico. - Motor 2.0 - 2 dueños. - 102.000 Km. - 6 airbags. - Todo funcionando bien. - En muy buenas condiciones, uso familiar. - Todo al día. - +56990132490 - DETALLES: - ⁠Detalles de pintura típicos por año y uso. Si quieres conocer todo nuestro stock vehicular visita el siguiente link. https://www.instagram.com/jyp_inversionistas/ - Se puede visitar todos los días previa llamada telefónica en La Florida. #jypinversionistas #concesionario #Seminuevos #santiago #Mazda3.","[al dia, detalles menores]",https://web.facebook.com/marketplace/item/1637449120414109/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [108]:
csv_name = f"{make}.csv"
# Guardar el DataFrame en un archivo CSV
items_df.to_csv(csv_name, index=False)  # El parámetro index=False evita que se agregue una columna de índice al archivo CSV

In [107]:
# Extraer los precios y URLs de la lista de diccionarios
prices = [item["Price"] for item in items_list]
urls = [item["URL"] for item in items_list]

# Calcular el precio promedio
precio_promedio = np.mean(prices)

# Calcular el precio más barato
precio_mas_barato = np.min(prices)
indice_precio_mas_barato = np.argmin(prices)

# Calcular el percentil 10 más bajo
percentil_10_mas_bajo = np.percentile(prices, 10)

# Calcular la mediana de los precios
mediana_precios = np.median(prices)


# Imprimir los resultados
print("Precio promedio:", precio_promedio)
print("Mediana de precios:", mediana_precios)
print("Percentil 10 más bajo:", percentil_10_mas_bajo)
print("Precio más barato:", precio_mas_barato)
print("URL:", urls[indice_precio_mas_barato])

Precio promedio: 5752996.091044776
Mediana de precios: 5800000.0
Percentil 10 más bajo: 2100000.0
Precio más barato: 500000
URL: https://web.facebook.com/marketplace/item/426718639853323/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
